In [2]:
!pip install -q fsrs_optimizer==4.18.2 

DEPRECATION: Loading egg at c:\users\moses\anaconda3\envs\ml_audio\lib\site-packages\sacremoses-0.0.43-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [3]:

import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import MinMaxScaler
from fsrs_optimizer import lineToTensor, FSRS
import torch
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
# Necessary Input:
# z_score - only for questions that have been asked
# asked questions to put into scheduler

# parameters for FSRS
w = [1.1008, 1.2746, 5.7619, 10.5114, 5.3148, 1.5796, 1.244, 0.003, 1.5741, 0.1741, 1.0137, 2.7279, 0.0114, 0.3071, 0.3981, 0.0, 1.9569]
requestRetention = 0.82  # recommended setting: 0.8 ~ 0.9

# parameters for Anki
graduatingInterval = 1
easyInterval = 4
easyBonus = 1.3
hardInterval = 1.2
intervalModifier = 1
newInterval = 0
minimumInterval = 1
leechThreshold = 8
leechSuspend = False

# common parameters
maximumInterval = 36500
new_cards_limits = 20
review_limits = 400
max_time_limts = 10000
learn_days = 50

# smooth curves
moving_average_period = 14

# Set it to True if you don't want the optimizer to use the review logs from suspended cards.
filter_out_suspended_cards = False

# Red: 1, Orange: 2, Green: 3, Blue: 4, Pink: 5, Turquoise: 6, Purple: 7
# Set it to [1, 2] if you don't want the optimizer to use the review logs from cards with red or orange flag.
filter_out_flags = []


def get_schedule_scores(df, lesson_id):
    '''Gets a df that contains id of question and schedule_scores for each one.
    Input takes the full df and the lesson_id, or the number of days since the beginning of the course.
    FSRS uses a calculated stability metric, which is how "stable" the idea is in your mind,
    and schedules the next occurence of the card. We then scale this and normalize to give an output number
    showing how urgent the card is, with 1 being the most urgent.
    This is simply a time-series model with Markov property:
    Depending on half-life, recall probability, result of recall, and difficulty, we define
    a memory state-transition equation to update at every step.
    We combine this with a Stochastic Shortest Path problem - the number of reviews required for memorizing something
    to the require half-life is uncertain, so we combine the SSP and MMC to
    find the optimal review time over iterations.'''
    deck_size = len(df)
    def calculate_review_duration(states, times):
        if states[-1] != 2:
            return 5
        else:
            # Find the most recent transition to state 2 from either 1 or 3
            for i in range(len(states) - 1, 0, -1):
                if states[i] == 2 and (states[i - 1] == 1 or states[i - 1] == 3):
                    return times[i] - times[i - 1]
            return 5  # Default value if no valid transition is found

    # Apply the function to each row
    df['review_duration'] = [calculate_review_duration(s, t) for s, t in zip(df['review_state'], df['review_time'])]

    # Define the bins for the intervals
    # bins = [0.25, 0.5, 0.75]
    # df['z_score_last'] = df['z_scores'].apply(lambda x: x[-1] if isinstance(x, list) and x else np.nan)
    # # Use numpy's digitize method to convert z_scores to review_rating
    # df['review_rating'] = np.digitize(df['z_score_last'], bins) + 1
    # df['review_rating'] = 5 - df['review_rating']

    def assign_rating(row):
        # Extract the last z_score value
        z_score = row['z_scores'][-1] if isinstance(row['z_scores'], list) and row['z_scores'] else None
        
        # Assign ratings based on z_score
        if z_score is not None:
            if z_score <= 0.25:
                return 1
            elif z_score <= 0.5:
                return 2
            elif z_score <= 0.75:
                return 3
            else:
                return 4
        else:
            return None  # or some default value
    df['review_rating'] = df.apply(assign_rating, axis=1) 
    df['review_time_curr'] = df['review_time'].apply(lambda x: x[-1])
    df['review_state_curr'] = df['review_state'].apply(lambda x: x[-1])
    New = 0
    Learning = 1
    Review = 2
    Relearning = 3

    df.sort_values(by=["id", "review_time_curr"], inplace=True, ignore_index=True)


    recall_card_revlog = df[
        (df["review_state_curr"] == Review) & (df["review_rating"].isin([2, 3, 4]))
    ]
    review_rating_prob = np.zeros(3)
    review_rating_prob[recall_card_revlog["review_rating"].value_counts().index - 2] = (
        recall_card_revlog["review_rating"].value_counts()
        / recall_card_revlog["review_rating"].count()
    )
    random_array = np.random.rand(4)
    random_array /= random_array.sum()
    first_rating_prob = random_array


    df["review_state_curr"] = df["review_state_curr"].map(
        lambda x: x if x != New else Learning)

    recall_costs = np.zeros(3)
    recall_costs_df = recall_card_revlog.groupby(by="review_rating")[
        "review_duration"
    ].mean()
    recall_costs[recall_costs_df.index - 2] = recall_costs_df / 1000

    state_sequence = np.array(df["review_state_curr"])
    duration_sequence = np.array(df["review_duration"])
    learn_cost = round(
        df[df["review_state_curr"] == Learning]["review_duration"].sum()
        / len(df["id"].unique())
        / 1000,
        1,
    )

    state_block = dict()
    state_count = dict()
    state_duration = dict()
    last_state = state_sequence[0]
    state_block[last_state] = 1
    state_count[last_state] = 1
    state_duration[last_state] = duration_sequence[0]
    for i, state in enumerate(state_sequence[1:]):
        state_count[state] = state_count.setdefault(state, 0) + 1
        state_duration[state] = state_duration.setdefault(
            state, 0) + duration_sequence[i]
        if state != last_state:
            state_block[state] = state_block.setdefault(state, 0) + 1
        last_state = state

    recall_cost = round(state_duration[Review] / state_count[Review] / 1000, 1)

    if Relearning in state_count and Relearning in state_block:
        forget_cost = round(
            state_duration[Relearning] /
            state_block[Relearning] / 1000 + recall_cost,
            1,
        )

    def generate_rating(review_type):
        if review_type == "new":
            return np.random.choice([1, 2, 3, 4], p=first_rating_prob)
        elif review_type == "recall":
            return np.random.choice([2, 3, 4], p=review_rating_prob)

    class Collection:
        def __init__(self):
            self.model = FSRS(w)
            self.model.eval()

        def states(self, t_history, r_history):
            with torch.no_grad():
                line_tensor = lineToTensor(
                    list(zip([str(t_history)], [str(r_history)]))[0]
                ).unsqueeze(1)
                output_t = self.model(line_tensor)
                return output_t[-1][0]

        def next_states(self, states, t, r):
            with torch.no_grad():
                return self.model.step(torch.FloatTensor([[t, r]]), states.unsqueeze(0))[0]

        def init(self, idx):
            t = df["review_time_curr"][idx]
            r = df["review_rating"][idx]
            p = round(first_rating_prob[r - 1], 2)
            new_states = self.states(t, r)
            return r, t, p, new_states


    feature_list = [
        "id",
        "difficulty",
        "stability",
        "retrievability",
        "delta_t",
        "reps",
        "lapses",
        "last_date",
        "due",
        "r_history",
        "t_history",
        "p_history",
        "states",
        "time",
        "factor",
    ]
    field_map = {key: i for i, key in enumerate(feature_list)}


    def fsrs4anki_scheduler(stability):
        def constrain_interval(stability):
            if stability > 0:
                return min(
                    max(1, round(9 * stability * (1 / requestRetention - 1))),
                    maximumInterval,
                )
            else:
                return 1

        interval = constrain_interval(stability)
        return interval


    def scheduler(fsrs_inputs):
            return fsrs4anki_scheduler(fsrs_inputs), 2.5

    #for scheduler_name in ("anki", "fsrs"):
    for scheduler_name in ["fsrs"]:
        new_card_per_day = np.array([0] * learn_days)
        new_card_per_day_average_per_period = np.array([0.0] * learn_days)
        review_card_per_day = np.array([0.0] * learn_days)
        review_card_per_day_average_per_period = np.array([0.0] * learn_days)
        time_per_day = np.array([0.0] * learn_days)
        time_per_day_average_per_period = np.array([0.0] * learn_days)
        learned_per_day = np.array([0.0] * learn_days)
        retention_per_day = np.array([0.0] * learn_days)
        expected_memorization_per_day = np.array([0.0] * learn_days)

        card = pd.DataFrame(
            np.zeros((deck_size, len(feature_list))),
            index=range(deck_size),
            columns=feature_list,
        )
        card["id"] = df["id"]
        card["states"] = card["states"].astype(object)
        card['reps'] = df['review_state'].apply(lambda x: len(x))
        card["lapses"] = 0
        card["due"] = learn_days
        card["last_date"] = df["review_time"].apply(lambda x: x[-1])
        
        student = Collection()
        random.seed(2022)
        # do 1 step:
        day = lesson_id
        reviewed = 0
        learned = 0
        review_time_today = 0
        learn_time_today = 0

        card["delta_t"] = day - card["last_date"]
        card["retrievability"] = np.power(
            1 + card["delta_t"] / (9 * card["stability"]), -1
        )
        need_learn = card[card["stability"] == 0]

        for idx in need_learn.index:
            if (
                learned >= new_cards_limits
                or review_time_today + learn_time_today >= max_time_limts
            ):
                break
            learned += 1
            learn_time_today += learn_cost
            #card.iat[idx, field_map["last_date"]] = day

            #card.iat[idx, field_map["reps"]] = 1
            #card.iat[idx, field_map["lapses"]] = 0

            r, t, p, new_states = student.init(idx)
            new_stability = float(new_states[0])
            new_difficulty = float(new_states[1])
            card['r_history'] = card['r_history'].astype(object)
            card['t_history'] = card['t_history'].astype(object)
            card['p_history'] = card['p_history'].astype(object)
            card.iat[idx, field_map["r_history"]] = str(r)
            card.iat[idx, field_map["t_history"]] = str(t)
            card.iat[idx, field_map["p_history"]] = str(p)
            card.iat[idx, field_map["stability"]] = new_stability
            card.iat[idx, field_map["difficulty"]] = new_difficulty
            card.iat[idx, field_map["states"]] = new_states

            delta_t, factor = scheduler(new_stability)
            card.iat[idx, field_map["due"]] = day + delta_t
            #card.iat[idx, field_map["due"]] = day + delta_t
            card.iat[idx, field_map["factor"]] = factor

            card.iat[idx, field_map["time"]] = learn_cost


        new_card_per_day[day] = learned
        review_card_per_day[day] = reviewed
        learned_per_day[day] = learned_per_day[day - 1] + learned
        time_per_day[day] = review_time_today + learn_time_today
        expected_memorization_per_day[day] = sum(
            card[card["retrievability"] > 0]["retrievability"]
        )

        if day >= moving_average_period:
            new_card_per_day_average_per_period[day] = np.true_divide(
                new_card_per_day[day - moving_average_period: day].sum(),
                moving_average_period,
            )
            review_card_per_day_average_per_period[day] = np.true_divide(
                review_card_per_day[day - moving_average_period: day].sum(),
                moving_average_period,
            )
            time_per_day_average_per_period[day] = np.true_divide(
                time_per_day[day - moving_average_period: day].sum(),
                moving_average_period,
            )
        else:
            new_card_per_day_average_per_period[day] = np.true_divide(
                new_card_per_day[: day + 1].sum(), day + 1
            )
            review_card_per_day_average_per_period[day] = np.true_divide(
                review_card_per_day[: day + 1].sum(), day + 1
            )
            time_per_day_average_per_period[day] = np.true_divide(
                time_per_day[: day + 1].sum(), day + 1
            )
    scaler = MinMaxScaler(feature_range=(0, 1))
    card['schedule_score'] = scaler.fit_transform(card[['due']])

    # Inverting the values so that lower 'due' values are closer to 1
    card['schedule_score'] = 1 - card['schedule_score']

    # Returning the DataFrame with 'id' and 'schedule_score'
    result = card[['id', 'schedule_score']]
    return result

c:\Users\Moses\anaconda3\envs\ml_audio\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = pd.read_pickle('data_full.pkl')
df.head()

,id,question,options,_category,embedding,review_state,review_time,z_scores,s_score,Q_score
0,0,Sumita Khatri wrote a research report and foll...,"[No., Yes, relating to performance presentatio...",Ethical and Professional Standards,"[0.007968312129378319, -0.0021401941776275635,...","[0, 1, 2, 3, 2, 3, 2, 3, 2, 3, 2]","[15, 27, 14, 27, 9, 29, 25, 17, 21, 27, 10]","[0.3820518737576567, 0.5363807206058436, 0.843...",0.610634,0.068177
1,1,"Aryana Reid, CFA, is a private wealth manager....",[Publish the post and send it as a mail to the...,Ethical and Professional Standards,"[-0.02464733086526394, -0.014164824038743973, ...","[0, 1, 2, 3, 2, 3]","[12, 11, 19, 15, 10, 29]","[0.7689862117474828, 0.786084508687689, 0.2121...",0.296293,0.873316
2,2,Kent Miller is an investment adviser at UBN In...,"[determine Hartford’s needs, objectives, and t...",Ethical and Professional Standards,"[-0.0013949209824204445, -0.01329685840755701,...","[0, 1, 2, 3, 2, 3, 2, 3, 2]","[21, 17, 10, 14, 9, 4, 1, 25, 22]","[0.870843353639129, 0.2345210061401538, 0.3750...",0.962313,0.997442
3,3,"Justin Zoghlin, CFA, was hired as a wealth man...","[Serving the religious community., Managing no...",Ethical and Professional Standards,"[-0.01206973847001791, -0.030683474615216255, ...","[0, 1, 2, 3, 2, 3, 2, 3, 2]","[11, 1, 18, 10, 8, 15, 27, 3, 8]","[0.4614002670673235, 0.35638086787218004, 0.90...",0.036187,0.984961
4,4,"Rani Kaporwala, CFA, an analyst at Smart Secur...","[No., Yes, relating to communications with cli...",Ethical and Professional Standards,"[0.016661042347550392, -0.013284368440508842, ...","[0, 1, 2, 3, 2, 3, 2, 3, 2]","[19, 19, 29, 22, 4, 21, 25, 10, 19]","[0.16358718871336497, 0.010827304568776674, 0....",0.635618,0.141168


In [5]:
result = get_schedule_scores(df, 0)

In [6]:
print(result)

    id  schedule_score
0    0    8.125000e-01
1    1    8.125000e-01
2    2    1.000000e+00
3    3    9.791667e-01
4    4    1.000000e+00
5    5    6.041667e-01
6    6    1.000000e+00
7    7    1.000000e+00
8    8    8.125000e-01
9    9    6.041667e-01
10  10    8.125000e-01
11  11    1.000000e+00
12  12    6.041667e-01
13  13    8.125000e-01
14  14    1.000000e+00
15  15    8.125000e-01
16  16    9.791667e-01
17  17    8.125000e-01
18  18    9.791667e-01
19  19    8.125000e-01
20  20    1.110223e-16
21  21    1.110223e-16
22  22    1.110223e-16
23  23    1.110223e-16
24  24    1.110223e-16
25  25    1.110223e-16
26  26    1.110223e-16
27  27    1.110223e-16
28  28    1.110223e-16
29  29    1.110223e-16
30  30    1.110223e-16
31  31    1.110223e-16
32  32    1.110223e-16
33  33    1.110223e-16


In [15]:
import json
from ast import literal_eval
from sklearn.manifold import TSNE
from flask import jsonify

df = pd.read_pickle('data_full.pkl')
def prepare_data(data):
    matrix = np.array(data['embedding'].apply(json.dumps).apply(literal_eval).to_list())
    tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
    vis_dims = tsne.fit_transform(matrix)

    # Preparing the response data
    response_data = []
    for idx, (x_coord, y_coord) in enumerate(vis_dims):
        response_data.append({
            "x": x_coord,
            "y": y_coord,
            "zScore": data['z_scores'][idx][-1],  # Assuming the last entry in each array
            "category": data['_category'][idx]
        })

    return response_data

def visualise():
    response_data = prepare_data(df)
    return response_data



In [16]:
print(visualise())

[{'x': -20.537004, 'y': -208.73245, 'zScore': 0.526107412529392, 'category': 'Ethical and Professional Standards'}, {'x': -57.00378, 'y': -125.275246, 'zScore': 0.5450016460374988, 'category': 'Ethical and Professional Standards'}, {'x': -62.93753, 'y': -177.18803, 'zScore': 0.06186905386364783, 'category': 'Ethical and Professional Standards'}, {'x': 62.372486, 'y': -191.55228, 'zScore': 0.4003930641352953, 'category': 'Ethical and Professional Standards'}, {'x': -26.640646, 'y': -156.16054, 'zScore': 0.012726102658469962, 'category': 'Ethical and Professional Standards'}, {'x': 65.39221, 'y': -132.69086, 'zScore': 0.9357106251384566, 'category': 'Ethical and Professional Standards'}, {'x': 25.448769, 'y': -214.51826, 'zScore': 0.19335374220252655, 'category': 'Ethical and Professional Standards'}, {'x': 16.963383, 'y': -158.80453, 'zScore': 0.0363450151180138, 'category': 'Ethical and Professional Standards'}, {'x': -0.9503921, 'y': -56.66005, 'zScore': 0.6204247831994427, 'category'